# File Used For Predicting Next day Model

## Loading whole dataset

In [1]:
import features as ft
from datetime import date, timedelta
import pandas as pd
import xgboost as xgb

c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:21: SyntaxWarning: "\$" is an invalid escape sequence. Such sequences will not work in the future. Did you mean "\\$"? A raw string is also an option.
  .str.replace('[\$,\+\-\%>]', '', regex=True)


In [ ]:

'''
df = joblib.load('saved_vars/35_features_2015_21-10-2025.joblib')
print('/nData loaded in from joblib file./n')
df = ft.update_latest_data(df)

df_by_ticker = df.copy()
print(df_by_ticker.shape)
#remove rows with nans or neg nums in certain rows.
df_by_ticker = df_by_ticker.dropna()
df_by_ticker = df_by_ticker[(df_by_ticker[['last_price', 'Qty', 'shares_held', 'Owned', 'Value', 'Owned_norm', 'title_rank', 'filing_lad_days', 'same_day_trade_count']] >= 0).all(axis=1)]
print(df_by_ticker.shape)
'''

"\ndf = joblib.load('saved_vars/35_features_2015_21-10-2025.joblib')\nprint('/nData loaded in from joblib file./n')\ndf = ft.update_latest_data(df)\n\ndf_by_ticker = df.copy()\nprint(df_by_ticker.shape)\n#remove rows with nans or neg nums in certain rows.\ndf_by_ticker = df_by_ticker.dropna()\ndf_by_ticker = df_by_ticker[(df_by_ticker[['last_price', 'Qty', 'shares_held', 'Owned', 'Value', 'Owned_norm', 'title_rank', 'filing_lad_days', 'same_day_trade_count']] >= 0).all(axis=1)]\nprint(df_by_ticker.shape)\n"

In [3]:
#todays_trades = df_by_ticker[df_by_ticker['eff_trans_date'] == df_by_ticker.iloc[-1]['eff_trans_date']]
#df_by_ticker = df_by_ticker[df_by_ticker['eff_trans_date'] != df_by_ticker.iloc[-1]['eff_trans_date']]
#df_by_ticker.shape

## Loading Todays Trades.

In [10]:
yesterday = date.today() - timedelta(days=1)
print(yesterday)

todays_trades = ft.pull_data(startdate=str(yesterday), filtered=False)

2025-10-23 21:17:27,759 - openinsider - INFO - Starting scraping process...
2025-10-23 21:17:27,759 - openinsider - INFO - Starting scraping process...


2025-10-22
Scraping data


Processing months: 100%|██████████| 1/1 [00:00<00:00, 953.90it/s]
2025-10-23 21:17:27,765 - openinsider - INFO - Scraping completed. Found 294 transactions.
2025-10-23 21:17:27,765 - openinsider - INFO - Scraping completed. Found 294 transactions.
2025-10-23 21:17:27,768 - openinsider - INFO - Data saved to data\insider_trades.csv
2025-10-23 21:17:27,768 - openinsider - INFO - Data saved to data\insider_trades.csv


Cleaning data


c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:78: FutureWarning: YF.download() has changed argument auto_adjust default to True
  downloaded_prices = yf.download(
[*********************100%***********************]  146 of 146 completed

4 Failed downloads:
['VHAIO', 'LKSP', 'PHXE.', 'DMII']: YFTzMissingError('possibly delisted; no timezone found')


Adding features, this may take a while.
Starting create features


c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:135: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())
c:\Users\azzag\Documents\Programming\Python\TradingBot\features.py:135: RuntimeWarning: invalid value encountered in scalar divide
  (ticker_data['Volume'].iloc[-1] - ticker_data['Volume'].mean())


Finish create features
Starting count_same_day_trades
Finished count_same_day_trades


In [11]:
print(yesterday)
todays_trades
todays_trades = todays_trades[todays_trades['eff_trans_date'] >= pd.to_datetime(yesterday)] #scrape() rounds down to the first of the month, this ensures only new trades are merged.
todays_trades

2025-10-22


,transaction_date,trade_date,ticker,company_name,owner_name,Title,transaction_type,last_price,Qty,shares_held,...,price_vs_sma50,day_sin,day_cos,filing_lad_days,title_rank,SPY_1d_return,is_during_market_hours,month_end_flag,Owned_norm,same_day_trade_count
288,2025-10-22 12:58:36,2025-10-21,TPL,Texas Pacific Land Corp,Horizon Kinetics Asset Management LLC,10%,P - Purchase,938.73,1.000000e+00,1.162548e+06,...,1.006388,0.974928,-0.222521,1.0,1,-0.00625,True,0,0.000000,1
289,2025-10-22 14:19:28,2025-10-22,NUGN,"Livento Group, Inc.",Henriksen Michael Gregory Max,"Chief Investment Officer, 10%",P - Purchase,0.00,2.000000e+09,2.147484e+09,...,0.485437,0.974928,-0.222521,0.0,3,-0.00625,True,0,0.722608,1
290,2025-10-22 15:28:04,2025-10-20,MNTR,"Mentor Capital, Inc.",Billingsley Chester,"CEO, 10%",P - Purchase,0.10,6.000000e+04,3.128296e+06,...,1.262626,0.974928,-0.222521,2.0,4,-0.00625,True,0,0.013767,1
291,2025-10-22 16:02:34,2025-10-20,FITB,Fifth Third Bancorp,Bayh Evan,Dir,P - Purchase,41.22,3.000000e+03,8.494200e+04,...,0.948497,0.974928,-0.222521,2.0,2,-0.00625,False,0,0.027345,1
292,2025-10-22 16:11:49,2025-04-30,EPAM,"Epam Systems, Inc.",Yezhkov Sergey,"SVP, Co-Head of Global Busines",P - Purchase,133.37,5.600000e+01,1.173600e+04,...,0.990122,0.974928,-0.222521,175.0,3,-0.00625,False,1,0.000000,1
293,2025-10-22 17:13:12,2025-10-20,CNS,"Cohen & Steers, Inc.",Steers Robert Hamilton,"Exec COB, 10%",P - Purchase,70.21,4.053900e+04,1.186479e+07,...,0.981547,0.974928,-0.222521,2.0,3,-0.00625,False,0,0.000000,1


### Predicting tomorrows movements.

In [12]:
clf = xgb.XGBClassifier()
clf.load_model("saved_vars/XGBoost/23_10_2025_none_rand_spli_accuracy_0.85_threshold_0.728.json")
chosen_threshold = 0.728

In [13]:
todays_trades_dropped = todays_trades.drop([
    'transaction_date', 'trade_date', 'eff_trans_date', 
    'ticker','company_name', 'owner_name', 'prev_prices', 'future_prices', 
    'Title', 'transaction_type', 'Owned','trade_date_epoch', 'eff_trans_date_epoch',
    'last_price'
    ], axis=1)

#display(todays_trades_dropped)
todays_trades_dropped = todays_trades_dropped[clf.feature_names_in_]
todays_trades_proba = clf.predict_proba(todays_trades_dropped)[:, 1]
todays_trades_pred = (todays_trades_proba > chosen_threshold).astype(int)

print(todays_trades_proba)
print(todays_trades_pred)
display(todays_trades[todays_trades_pred==1])

[0.14745557 0.6063846  0.86907166 0.00756929 0.05925268 0.19889484]
[0 0 1 0 0 0]


,transaction_date,trade_date,ticker,company_name,owner_name,Title,transaction_type,last_price,Qty,shares_held,...,price_vs_sma50,day_sin,day_cos,filing_lad_days,title_rank,SPY_1d_return,is_during_market_hours,month_end_flag,Owned_norm,same_day_trade_count
290,2025-10-22 15:28:04,2025-10-20,MNTR,"Mentor Capital, Inc.",Billingsley Chester,"CEO, 10%",P - Purchase,0.1,60000.0,3128296.0,...,1.262626,0.974928,-0.222521,2.0,4,-0.00625,True,0,0.013767,1
